In [1]:
import numpy as np

In [2]:
def post_money_safe(val, inv):
    return inv / val * 100

def pre_money_safe(pre, stake, inv):
    return stake * pre / (pre+inv)


# B brings in 10MM on a 100MM post money valuation on ABC
# how much B owns?

post_val = 100
new_inv = 10

b_post_stake = post_money_safe(post_val, new_inv)
print(b_post_stake)

# Investor A had 10% stake on ABC (10MM investment with a 100MM pre money Valuation)
# B brings in an additional 10MM on a new round
# how much A still have?

a_pre_stake = 50
pre_val = 90

a_post_stake = pre_money_safe(pre_val, a_pre_stake, new_inv)
print(a_post_stake)

10.0
45.0


In [3]:
class Investment:

    def __init__(self, invested, round):
        self.invested = invested
        self.round = round
        self.shares = self.invested / self.round.price

class Investor:

    def __init__(self, name):
        self.name = name
        self.investments = []

    def new_investment(self, invested, round):
        investment = Investment(invested, round)
        self.investments.append(investment)

    def total_shares(self):
        shares = 0
        for i in self.investments:
            shares += i.shares 
        return shares
    
    def total_invested(self):
        invested = 0
        for i in self.investments:
            invested += i.invested 
        return invested


class Covenants:
    
    def __init__(self, covenants={}):
        self.liquidity_preference = covenants.get('liquidity_preference', 0)
           
class Round:

    def __init__(self, name, invested, valuation, shares, covenants):
        self.name = name
        self.invested = invested
        self.valuation = valuation
        self.shares = shares
        self.price = valuation/shares
        self.covenants = covenants

class Deal:

    def __init__(self, investment, stake):
        self.investment = investment
        self.stake = stake / 100

class Company:
    
    def __init__(self, name, equity, shares):
        self.name = name

        self.incorporation = Round('incorporation', equity, equity, shares, Covenants())

        self.founder = Investor('founder')
        self.founder.new_investment(equity, self.incorporation)

        self.investors = [self.founder]
        self.rounds = [self.incorporation]

    def current_stake(self):
        cap = []
        for i in self.investors:
            last_round = self.rounds[-1]
            total_shares = last_round.shares
            investor_shares = i.total_shares()
            cap.append({
                'investor': i.name,
                'stake': investor_shares / total_shares * 100
            })
        return cap

    def get_investor(self, name):
        for i in self.investors:
            if name == i.name:
                return i
        return None

    def add_investor(self, name):
        new_investor = Investor(name)
        self.investors.append(new_investor)
        return new_investor

    def get_add_investor(self, name):
        investor = self.get_investor(name)
        if investor is None:
            investor = self.add_investor(name)
        return investor

    def new_round(self, name, investors, deal, covenants):

        post_money_valuation = deal.investment / deal.stake
        pre_money_valuation = post_money_valuation -  deal.investment
        
        total_shares =  self.rounds[-1].shares * ( 1 + deal.investment / pre_money_valuation)
        
        round = Round(name, deal.investment, post_money_valuation, total_shares, covenants = covenants)
        
        self.rounds.append(round)

        for i in investors:
            new_investor = self.get_add_investor(i['name'])
            new_investor.new_investment(deal.investment * (i['stake']/100), round)
            
        return None

In [4]:
capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)
company.incorporation.valuation

10000.0

In [5]:
cap = company.current_stake()
cap

[{'investor': 'founder', 'stake': 100.0}]

In [6]:
deal = Deal(10000, 10)
investors = [{'name': 'ze', 'stake': 100}]
company.new_round('seed', investors, deal, Covenants({}))

deal = Deal(500000, 20)
investors = [{'name': 'tera', 'stake': 80}, {'name': 'patria', 'stake': 20}]
covenants = Covenants({'liquidity_preference': 2})
company.new_round('a', investors, deal, covenants)

In [7]:
company.current_stake()

[{'investor': 'founder', 'stake': 72.0},
 {'investor': 'ze', 'stake': 8.000000000000002},
 {'investor': 'tera', 'stake': 16.0},
 {'investor': 'patria', 'stake': 4.0}]

In [86]:
exit_multiple = 4
last_valuation = company.rounds[-1].valuation 
total_shares = company.rounds[-1].shares
exit_valuation = last_valuation*exit_multiple
print('---------------------------------------------')
print('total shares: %s' % total_shares)
print('last valuation: %s' % last_valuation)
print('exit valuation: %s' % exit_valuation)
print('exit price: %s' % (exit_valuation / total_shares))
print('---------------------------------------------')
print('investor', '|', 'round', '|', 'round_size', '|', 'invested', '|', 'stakes', '|', 'round_stake', '|', 'liquidity_preference')
for i in company.investors:
    for a in i.investments:
        print(i.name, '|', a.round.name, '|', a.round.invested, '|', a.invested, '|', a.shares/total_shares*100, '|', a.invested/a.round.invested*100, '|', a.round.covenants.liquidity_preference )
print('---------------------------------------------')

---------------------------------------------
total shares: 1388888.888888889
last valuation: 2500000.0
exit valuation: 10000000.0
exit price: 7.199999999999999
---------------------------------------------
investor | round | round_size | invested | stakes | round_stake | liquidity_preference
founder | incorporation | 10000.0 | 10000.0 | 72.0 | 100.0 | 0
ze | seed | 10000 | 10000.0 | 8.000000000000002 | 100.0 | 0
tera | a | 500000 | 400000.0 | 16.0 | 80.0 | 2
patria | a | 500000 | 100000.0 | 4.0 | 20.0 | 2
---------------------------------------------


In [87]:
val_to_dist = exit_valuation
aux_investments = []
for i in company.investors:
    for a in i.investments:
        aux_investment = {
            'name': i.name,
            'round': a.round.name,
            'invested': a.invested,
            'stake': a.shares/total_shares,
            'round_size': a.round.invested,
            'round_share': a.invested/a.round.invested,
            'liq_pref': a.round.covenants.liquidity_preference,
            'stake_value': a.shares/total_shares*exit_valuation,
            'liq_pref_floor':  a.invested * a.round.covenants.liquidity_preference
        }
        aux_investments.append(aux_investment)

for r in company.rounds[::-1]:
    r_invested = r.invested
    r_liq_pref = r.covenants.liquidity_preference
    r_to_pay = np.minimum(r_invested * r_liq_pref,val_to_dist) if r_liq_pref > 0 else val_to_dist
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name:
            stake_value = i['stake']*r_to_pay
            liquidity_pref_floor = np.minimum(i['invested']*i['liq_pref'], r_to_pay * i['round_share'])
            paid_due_to_pref = np.maximum(liquidity_pref_floor-stake_value, 0)
            aux_investments[k]['paid_due_to_pref']  = paid_due_to_pref
            val_to_dist -= paid_due_to_pref
    for k,i in enumerate(aux_investments):
        if i['round'] == r.name:
            paid_due_to_stake = i['stake'] * val_to_dist
            aux_investments[k]['paid_due_to_stake'] = paid_due_to_stake
            aux_investments[k]['total_paid'] =  paid_due_to_stake + i['paid_due_to_pref'] 
            aux_investments[k]['paid_due_to_pref'] = aux_investments[k]['total_paid']-i['stake_value']


In [88]:
print(aux_investments)

[{'name': 'founder', 'round': 'incorporation', 'invested': 10000.0, 'stake': 0.72, 'round_size': 10000.0, 'round_share': 1.0, 'liq_pref': 0, 'stake_value': 7200000.0, 'liq_pref_floor': 0.0, 'paid_due_to_pref': -576000.0, 'paid_due_to_stake': 6624000.0, 'total_paid': 6624000.0}, {'name': 'ze', 'round': 'seed', 'invested': 10000.0, 'stake': 0.08000000000000002, 'round_size': 10000, 'round_share': 1.0, 'liq_pref': 0, 'stake_value': 800000.0000000001, 'liq_pref_floor': 0.0, 'paid_due_to_pref': -64000.0, 'paid_due_to_stake': 736000.0000000001, 'total_paid': 736000.0000000001}, {'name': 'tera', 'round': 'a', 'invested': 400000.0, 'stake': 0.16, 'round_size': 500000, 'round_share': 0.8, 'liq_pref': 2, 'stake_value': 1600000.0, 'liq_pref_floor': 800000.0, 'paid_due_to_pref': 512000.0, 'paid_due_to_stake': 1472000.0, 'total_paid': 2112000.0}, {'name': 'patria', 'round': 'a', 'invested': 100000.0, 'stake': 0.04, 'round_size': 500000, 'round_share': 0.2, 'liq_pref': 2, 'stake_value': 400000.0, 'l